In [1]:
import numpy as np # linear algebra
#import matplotlib.pyplot as plt
import numba
from numba import float64 as f64
#import ipywidgets as widgets 
from ipywidgets import FloatSlider, FloatLogSlider, HBox, VBox
import matplotlib.ticker as mtick
import bqplot.pyplot as plt
from bqplot import DateScale, LinearScale, Lines, Axis, Figure

# MBBEFD

In [2]:
@numba.vectorize
def nb_clip(x, a, b):
    # numba doesn't support np.clip yet...
    if   x > b: return b
    elif x < a: return a
    else:       return x

# ===============================================
# Define the MBBEFD distribution, without class
# ===============================================

##remove class
#replace X1,X2 with c and mpl values (SI already included) and replace functions of X1,X2 with new functions (for ilf,mpl and mean)

@numba.vectorize([f64(f64, f64, f64, f64)])
def _mbbefd_cdf(x, b, g, mpl):
    # TODO: test all cases...
    if (np.isnan(x) or np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan
    x /= mpl
    if   x>=1:                    return 1
    elif g==1 or b==0 or x==0:    return 0
    elif b==1 and g>1:            return 1 - 1/(1+(g-1)*x)
    elif (b*g)==1 and g>1:        return 1 - b**x 
    elif b>0 and b and (b*g)!=1 and g>1: return 1 - (1 - b) / ((g - 1) * (b ** (1 - x)) + (1 - g * b))
    else: return np.nan

@numba.vectorize([f64(f64, f64, f64, f64)])
def _mbbefd_ppf(q, b, g, mpl):
    # TODO: test all cases...
    
    if (np.isnan(q) or np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan

    if q <= 0: return 0
    if q >= 1: return mpl # TODO: check
    return mpl * (1 - np.log( ((1-b)/(1-q) - 1 + g*b) / (g-1) ) / np.log(b))

@numba.vectorize([f64(f64, f64, f64, f64)])
def _mbbefd_ilf(x, b, g, mpl):
    # TODO: test all cases...
    
    if (np.isnan(x) or np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan

    if x > mpl: return 0 # TODO: check
    res = np.log(((g-1)*b + (1 - g*b) * (b**(x/mpl))) / (1 - b)) / np.log(g*b)
    return nb_clip(res, 0, 1) # TODO: check, mpl?\

@numba.vectorize([f64(f64, f64, f64)])
def _mbbefd_mean(b, g, mpl):
    # TODO: test
    if (np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan
    return mpl * (np.log(g * b) * (1 - b) / np.log(b) / (1 - g * b))


@numba.njit
def MBBEFD_cdf(x,c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_cdf(x, b, g, mpl)

@numba.njit
def MBBEFD_ilf(x,c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_ilf(x, b, g, mpl)

@numba.njit
def MBBEFD_ppf(q,c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_ppf(q, b, g, mpl)

@numba.njit
def MBBEFD_mean(c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_mean(b, g, mpl)

@numba.njit
def MBBEFD_c(c):
    b = np.exp(3.1 - 0.15 * (1 + c) * c)
    g = np.exp((0.78 + 0.12 * c) * c)
    return b,g

## Distributions of MBBEFD

In [3]:
# plot the MBBEFD curves
x = np.linspace(0, 1, 100)
cs = np.array([5,4,3,2,1])
b,g=5,10

#1st chart
title_tmpl_1 = 'MBBEFD cdf(b={:.5f} and g={:.5f})'
fig1 = plt.figure(title=title_tmpl_1.format(b,g))
#plt.xlabel("Damage Factor")
#plt.ylabel("cumulative probability")

ys1 = [MBBEFD_cdf(x,c,1) for c in cs]
line_chart_1 = plt.plot(x=x, y=[*ys1], stroke_width=1,labels=["c=5","c=4","c=3","c=2","c=1"])
line_chart_1 = plt.plot(x=x, y=_mbbefd_cdf(x,b,g,1),stroke_width=3)
#line_chart_1.tooltip = Tooltip(fields=["x", "y"], labels=["Damage Factor", "cdf"])

#2nd chart
title_tmpl_2 = 'ilf (exposure curve) (b={:.5f} and g={:.5f})'
fig2 = plt.figure(title=title_tmpl_2.format(b,g))
plt.xlabel("Damage Factor")
plt.ylabel("premium share (of fgu premium)")

ys2 = [MBBEFD_ilf(x,c,1) for c in cs]
line_chart_2 = plt.plot(x=x, y=[*ys2], stroke_width=1, display_legend=True)
line_chart_2 = plt.plot(x=x, y=_mbbefd_ilf(x,b,g,1),stroke_width=3)

# fig2.Legend_Location='bottom_right'
# #line_chart.tooltip = Tooltip(fields=["x", "y"], labels=["Damage Factor", "premium share"])





In [4]:
# use two sliders to represent mu and sigma
b_slider = FloatSlider(description='b', value=5, min=1.0001, max=100, step=.1)
g_slider = FloatLogSlider(description='g',value=10,base=10,
    min=0, # min exponent of base
    max=3, # max exponent of base,
    step=.1)

slider_layout = HBox([b_slider, g_slider])

In [5]:
def update_chart(change):
    b=b_slider.value
    g=g_slider.value
    
    #1st chart
    line_chart_1.y = _mbbefd_cdf(x,b,g,1)
    line_chart_1.stroke_width= 3
     
    # also update the fig title
    fig1.title = title_tmpl_1.format(b, g)
      
    #2nd chart
    line_chart_2.y = _mbbefd_ilf(x,b,g,1)
    line_chart_2.stroke_width= 3
    fig2.title = title_tmpl_2.format(b, g)
    
    #fig=fig1.append(fig2)
    
# register the above callback with the 'value' trait of the sliders
b_slider.observe(update_chart, 'value')
g_slider.observe(update_chart, 'value')   
    

In [6]:
# now put all the widgets together into a simple dashboard
# the plot should update now when the slider values are updated!
final_layout = VBox([fig1,fig2, slider_layout])
final_layout